In [2]:
import pandas as pd
import os
import numpy as np

import matplotlib.pyplot as plt
plt.set_cmap('gray')
plt.rcParams['figure.figsize'] = (15, 15)

from tqdm import tqdm_notebook as tqdm
import subtle.utils.io as suio
import subtle.subtle_preprocess as supre
from scipy.misc import imresize

/home/srivathsa/projects/SubtleGad/train/subtle/utils/hyperparameter.py:8: UserWarning: Module test_tube not found - hyperparameter related functions cannot be used
  warnings.warn('Module test_tube not found - hyperparameter related functions cannot be used')


<Figure size 432x288 with 0 Axes>

In [3]:
df_failure = pd.read_csv('/home/srivathsa/projects/studies/gad/failure_review/failure_mode.csv')
fig_path = '/home/srivathsa/projects/studies/gad/failure_review/plots'
dict_rows = []

for idx, df_row in tqdm(df_failure.iterrows(), total=len(df_failure.index)):
    inf_path = df_row['Inference Path']
    inf_id = inf_path.split('/')[-1]
    
    pp_path = inf_path.replace('/{}'.format(inf_id), '').replace('data', 'preprocess/data')
    pp_path = '{}/{}.h5'.format(pp_path, df_row['Case'])
    
    if not os.path.exists(pp_path):
        pp_path = pp_path.replace('h5', 'npy')
    
    imgs = suio.load_file(pp_path)
    if imgs.ndim == 5:
        imgs = imgs[0]
    
    imgs = imgs.transpose(1, 0, 2, 3)
    
    pre, low, full = imgs[:3]
    
    inf_path = '{}/{}/{}_SubtleGad'.format(inf_path, df_row['Case'], df_row['Case'])

    gad, _ = suio.dicom_files(inf_path)
    gad = np.interp(gad, (gad.min(), gad.max()), (full.min(), full.max()))
    
    if 'Prisma' in df_row['Case']:
        gad = supre.zero_pad(gad, target_size=256)
    
    sl_info = df_row['Slice']
    if str(sl_info) != 'nan':
        if ',' in sl_info:
            sl_info = sl_info.split(', ')[-1]
        orient, sl_idx = sl_info.split(' ')
        sl_idx = int(sl_idx)
    else:
        orient = ''
        sl_idx = pre.shape[0] // 2
    
    if ('NO' in df_row['Case'] or 'Prisma' in df_row['Case']) and orient == 'AX':
        p_sl = np.rot90(pre[:, sl_idx], k=3)
        l_sl = np.rot90(low[:, sl_idx], k=3)
        f_sl = np.rot90(full[:, sl_idx], k=3)
        g_sl = np.rot90(gad[:, sl_idx], k=3)
    else:
        p_sl = pre[sl_idx]
        l_sl = low[sl_idx]
        f_sl = full[sl_idx]
        g_sl = gad[sl_idx]
    
    if f_sl.shape[0] != g_sl.shape[0]:
        g_sl = imresize(g_sl, (f_sl.shape[0], f_sl.shape[1]))
    
    g_sl = np.interp(g_sl, (g_sl.min(), g_sl.max()), (f_sl.min(), f_sl.max()))
    row1 = np.hstack([p_sl, l_sl])
    row2 = np.hstack([f_sl, g_sl])
    row3 = np.hstack([np.abs(l_sl-p_sl), np.abs(f_sl-p_sl)])
    disp = np.vstack([row1, row2, row3])
    
    mode = 'F' if df_row['Status'] == 'Failure' else 'PS'
    plt.imshow(disp)
    plt.axis('off')
    plt.savefig('{}/{}_{}.png'.format(fig_path, df_row['Case'], mode))
    plt.clf()

/home/srivathsa/miniconda3/envs/py35gad/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/42 [00:00<?, ?it/s]

/home/srivathsa/miniconda3/envs/py35gad/lib/python3.6/site-packages/ipykernel_launcher.py:53: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


<Figure size 1080x1080 with 0 Axes>